# 成為初級資料分析師 | Python 與資料科學應用

> pandas 101：處理表格式資料的 Python 模組

## 郭耀仁

## 大綱

- pandas 解決了什麼問題
- pandas 基礎
- 奧運獎牌排行
- pandas 技巧
- 美國普查

## pandas 解決了什麼問題

## 隨堂練習：請計算註冊於開曼群島的上市公司股價中位數

<https://tw.stock.yahoo.com/d/i/rank.php?t=pri&e=tse&n=100>

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_price_rank():
    page_url = "https://tw.stock.yahoo.com/d/i/rank.php?t=pri&e=tse&n=100"
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text)
    stock_tickers = []
    stock_names = []
    for i in soup.select(".name a"):
        stock_ticker = i.text.split()[0]
        stock_name = i.text.split()[1]
        stock_tickers.append(stock_ticker)
        stock_names.append(stock_name)
    prices = []
    for i in range(5, 5+10*100, 10):
        price = soup.find_all("table")[2].find_all("td")[0].find_all("td")[i].text
        prices.append(float(price))
    return stock_tickers, stock_names, prices

In [ ]:
stock_tickers, stock_names, prices = get_price_rank()
print(stock_tickers)
print(stock_names)
print(prices)

In [ ]:
from statistics import median

ky_prices = [price for stock_name, price in zip(stock_names, prices) if "KY" in stock_name]
print(median(ky_prices))

## Python 一直以來都非常適合資料處理，但她的分析能力很薄弱，`pandas` 的開發有助於補足 Python 資料分析的需求，讓使用者能夠在 Python 中執行完整的資料分析流程，而無需切換到 data-centric 的特定語言，如 R。

In [ ]:
import pandas as pd

df = pd.DataFrame()
df["ticker"] = stock_tickers
df["stock_name"] = stock_names
df["price"] = prices
df.head()

In [ ]:
df[df["stock_name"].str.contains("KY")]["price"].median()

## pandas 提供了新的資料結構

- Series 是具備索引的 ndarray
- DataFrame 能完美處理表格式資料（tabular data）
- Panel 能為 DataFrame 加入第三個維度（通常為時間）

## pandas 主要的應用場景

- 表格式資料的讀取
- 豐富的資料清理與分析函數
- 視覺化：包裝了常用的 matplotlib.pyplot 圖形

## pandas 基礎

## 什麼是 pandas？

> Flexible and powerful data analysis / manipulation library for Python, providing labeled data structures similar to R data.frame objects, statistical functions, and much more.

## pandas 的命名源自她的三個資料結構

- **Pan**el
- **Da**taFrame
- **S**eries

## 創建 Series

In [ ]:
import pandas as pd

numbers = [9, 23, 33, 91, 13]
players = ["Ron Harper", "Michael Jordan", "Scottie Pippen", "Dennis Rodman", "Luc Longley"]
ser = pd.Series(players)
ser

In [ ]:
ser.index = numbers
ser

## 創建 DataFrame

In [ ]:
df = pd.DataFrame()
df["number"] = numbers
df["player"] = players
df

In [ ]:
df.set_index("player")

## 以 `df.<TAB>` 觀察屬性與方法

In [ ]:
df.head()

In [ ]:
df.tail(3)

In [ ]:
df.index

In [ ]:
df.describe()

## 選擇變數 Select

In [ ]:
df["player"]

## 篩選觀測值 Filter

- `df.loc[]` 以標籤為準
- `df.iloc[]` 以位置為準
- 以布林陣列篩選

In [ ]:
df.index = ["PG", "SG", "SF", "PF", "C"]
df.loc[["SG", "SF", "PF"], ["number", "player"]]

In [ ]:
df.iloc[[1, 2, 3], [0, 1]]

In [ ]:
is_trio = df["number"].isin([23, 33, 91])
print(is_trio)
df[is_trio]

## 排序觀測值 Arrange

- `df.sort_index()`
- `df.sort_values()`

In [ ]:
df.sort_index()

In [ ]:
df.sort_values(by="number")

## 獲取資料的最大值最小值

- `df["col_name"].max()`
- `df["col_name"].min()`

## 獲取資料最大值最小值的索引值

- `df["col_name"].idxmax()`
- `df["col_name"].idxmin()`

## 隨堂練習：誰的背號數字最大？

In [ ]:
import pandas as pd

numbers = [9, 23, 33, 91, 13]
players = ["Ron Harper", "Michael Jordan", "Scottie Pippen", "Dennis Rodman", "Luc Longley"]
df = pd.DataFrame()
df["number"] = numbers
df["player"] = players
df

In [ ]:
max_number = df["number"].max()
df[df["number"] == max_number]["player"][3]

In [ ]:
df.set_index("player")["number"].idxmax()

## 隨堂練習：誰的背號數字最小？

In [ ]:
min_number = df["number"].min()
df[df["number"] == min_number]["player"][0]

In [ ]:
df.set_index("player")["number"].idxmin()

## 奧運獎牌排行

## 來源

[Coursera](https://www.coursera.org/) 的 [Introduction to Data Science in Python](https://www.coursera.org/learn/python-data-analysis/home/welcome) 課程作業

In [ ]:
import pandas as pd

df = pd.read_csv('https://storage.googleapis.com/py_ml_datasets/olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')

In [ ]:
df.head()

## 隨堂練習：哪個國家贏得的夏季奧運金牌數最多？

```python
def answer_one():
    """
    這個函數應該回傳一個文字，國家名
    """
    return "答案"
```

In [ ]:
def answer_one(df):
    return df["Gold"].idxmax()

answer_one(df)

## 隨堂練習：哪個國家夏季奧運與冬季奧運的金牌數差距數最大？

```python
def answer_two():
    """
    這個函數應該回傳一個文字，國家名
    """
    return "答案"
```

In [ ]:
def answer_two(df):
    diff = df["Gold"] - df["Gold.1"]
    diff_abs = diff.abs()
    return diff_abs.idxmax()

answer_two(df)

## 隨堂練習：哪個國家夏季奧運與冬季奧運的金牌數差距除以總金牌數的比例最大？（僅考慮至少有一個夏季金牌與一個冬季金牌的國家）

$$\text{Ratio} = \frac{\text{Summer Gold} - \text{Winter Gold}}{\text{Total Gold}}$$

```python
def answer_three():
    """
    這個函數應該回傳一個文字，國家名
    """
    return "答案"
```

In [ ]:
def answer_three(df):
    summer_gold = df["Gold"]
    winter_gold = df["Gold.1"]
    total_gold = df["Gold.2"]
    ratio = (summer_gold - winter_gold)/total_gold
    ratio_not_one = ratio[ratio < 1]
    return ratio_not_one.idxmax()

answer_three(df)

## 隨堂練習：計算 146 個國家的獎牌點數，金牌 3 點、銀牌 2 點、銅牌 1 點。

```python
def answer_four():
    """
    這個函數應該回傳一個 Series，長度為 146
    """
    return "答案"
```

In [ ]:
def answer_four(df):
    total_gold = df["Gold.2"]
    total_silver = df["Silver.2"]
    total_bronze = df["Bronze.2"]
    points = 3*total_gold + 2*total_silver + total_bronze
    return points

answer_four(df)[:10]

## pandas 技巧

## 不那麼基礎的 pandas 觀念

- 資料的輸入輸出
- Apply
- 摘要
- 分組
- 遺漏值處理
- 轉置
- 合併

## 資料的輸入輸出

- 關聯式資料庫表格
- .csv 檔案
- array of JSON 的 .json 檔案
- Excel 試算表 .xls 與 .xlsx

In [ ]:
import requests

aqi_url = "https://opendata.epa.gov.tw/ws/Data/AQI/?$format=json"
r = requests.get(aqi_url, verify=False)
aqi = pd.read_json(r.text)
aqi.head()

## Apply

將函數映射至資料框

In [ ]:
import numpy as np

def get_na(x):
    """getting np.nan from "" and "ND" """
    if x == "" or x == "ND":
        return np.nan
    else:
        return x

In [ ]:
aqi["PM2.5"] = aqi["PM2.5"].apply(get_na)
aqi["AQI"] = aqi["AQI"].apply(get_na)
aqi.head()

## 遺漏值處理

- pandas 主要以 `np.nan` 紀錄遺漏值
- `df.isna()`
- `df.dropna()`
- `df.fillna()`

In [ ]:
aqi["PM2.5"].isna()

In [ ]:
# how many missing values are there
aqi["PM2.5"].isna().sum()

In [ ]:
aqi.dropna()

In [ ]:
filled_val = 999
aqi.fillna(filled_val)

## 摘要

In [ ]:
aqi["County"].unique()

In [ ]:
aqi["AQI"] = aqi["AQI"].astype(float)
aqi["PM2.5"] = aqi["PM2.5"].astype(float)
print(aqi["AQI"].max())
print(aqi["PM2.5"].min())

## 分組

In [ ]:
aqi.groupby("County").mean()["PM2.5"]

## 轉置

- `pd.melt()`：寬轉長格式
- `df.pivot()`：長轉寬格式

In [ ]:
wide_df = aqi[["SiteName", "AQI", "PM2.5", "SO2"]]
wide_df = wide_df.sort_values("SiteName").reset_index(drop=True)
wide_df.head()

In [ ]:
long_df = pd.melt(wide_df, id_vars=["SiteName"], value_vars=["AQI", "PM2.5", "SO2"], var_name="AirQualityMetric", value_name="Value")
long_df = long_df.sort_values("SiteName")
long_df = long_df.reset_index(drop=True)
print(long_df.shape)

In [ ]:
long_df.head(15)

In [ ]:
pivot_df = long_df.pivot(index="SiteName", columns="AirQualityMetric", values="Value")
pivot_df = pivot_df.reset_index()
del pivot_df.columns.name
pivot_df.head()

## 合併

利用 `pd.merge()` 進行 SQL 風格的合併

In [ ]:
left = long_df.copy()
right = aqi[["SiteName", "Latitude", "Longitude"]]
pd.merge(left, right, on="SiteName")

## 美國普查

## 來源

[Coursera](https://www.coursera.org/) 的 [Introduction to Data Science in Python](https://www.coursera.org/learn/python-data-analysis/home/welcome) 課程作業

In [ ]:
import pandas as pd

census_df = pd.read_csv('https://storage.googleapis.com/py_ml_datasets/census.csv')
census_df.shape

In [ ]:
census_df.head()

## 隨堂練習：哪個州（state）的郡（county）數最多？

```python
def answer_one():
    """
    這個函數應該回傳一個文字，州名
    """
    return "答案"
```

In [ ]:
def answer_one(df):
    return df.groupby("STNAME")["CTYNAME"].count().idxmax()

answer_one(census_df)

## 隨堂練習：僅考慮每州（state）人口最多的三個郡（county）計算人口總和（CENSUS2010POP），哪三個州總和數最多？（請注意 SUMLEV 變數）

```python
def answer_two():
    """
    這個函數應該回傳一個清單，三個州名
    """
    return "答案"
```

In [ ]:
def answer_two(df):
    county_df = df[df["SUMLEV"] == 50]
    groupby_summary = county_df.groupby("STNAME")["CENSUS2010POP"].nlargest(3)
    groupby_summary_df = pd.DataFrame(groupby_summary).reset_index()
    ans = groupby_summary_df.groupby("STNAME")["CENSUS2010POP"].sum().nlargest(3).index
    return list(ans)

answer_two(census_df)

## 哪個郡（county）在 2010-2015 期間人口改變數量最高？（POPESTIMATE2010:POPESTIMATE2015 這六個變數）

提示：如果 6 年的人口數分別為 120, 80, 105, 100, 130, 120 則人口改變數量為 130-80 = 50

```python
def answer_three():
    """
    這個函數應該回傳一個文字，郡名
    """
    return "答案"
```

In [ ]:
def answer_three(df):
    county_df = df[df["SUMLEV"] == 50]
    county_df = county_df.set_index("CTYNAME")
    county_df = county_df.loc[:, "POPESTIMATE2010":"POPESTIMATE2015"]
    max_population = county_df.loc[:, "POPESTIMATE2010":"POPESTIMATE2015"].max(axis=1)
    min_population = county_df.loc[:, "POPESTIMATE2010":"POPESTIMATE2015"].min(axis=1)
    diff_population = max_population - min_population
    return diff_population.idxmax()

answer_three(census_df)

## 篩選出屬於 REGION 1 或 2、開頭名稱為 Washington 並且 POPESTIMATE2015 大於 POPESTIMATE2014 的郡（county）

```python
def answer_four():
    """
    這個函數應該回傳一個 DataFrame，外型為 5x2，
    變數名稱為 ['STNAME', 'CTYNAME']，
    索引值由小到大排列
    """
    return "答案"
```

In [ ]:
def answer_four(df):
    county_df = df[df["SUMLEV"] == 50]
    contains_washington = county_df["CTYNAME"].str.contains("Washington")
    region_filter = (county_df["REGION"] == 1) | (county_df["REGION"] == 2)
    pop_filter = county_df["POPESTIMATE2015"] > county_df["POPESTIMATE2014"]
    ans = county_df[contains_washington & region_filter & pop_filter][["STNAME", "CTYNAME"]]
    ans = ans.reset_index(drop=True)
    return ans

answer_four(census_df)

## 延伸閱讀

[pandas: powerful Python data analysis toolkit](http://pandas.pydata.org/pandas-docs/stable/)

## 作業

## 擷取 Avengers: Endgame (2019) 的上映日期列表，改以 `pandas` 回答這兩個問題：

- 最多的上映日期為哪一天？
- 有幾個國家在那天上映？

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def answer_one():
    r = requests.get("https://www.imdb.com/title/tt4154796/releaseinfo")
    soup = BeautifulSoup(r.text)
    countries = [i.text.strip() for i in soup.select(".release-date-item__country-name a")]
    release_dates = [i.text for i in soup.select(".release-date-item__date")]
    df = pd.DataFrame()
    df["country"] = countries
    df["release_date"] = release_dates
    grouped = df.groupby("release_date")
    return grouped["country"].count()

In [2]:
answer_one()

release_date
22 April 2019     1
23 April 2019     1
24 April 2019    31
25 April 2019    22
26 April 2019    14
28 April 2019     1
29 April 2019     1
Name: country, dtype: int64

In [3]:
answer_one().idxmax()

'24 April 2019'

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def answer_two():
    r = requests.get("https://www.imdb.com/title/tt4154796/releaseinfo")
    soup = BeautifulSoup(r.text)
    countries = [i.text.strip() for i in soup.select(".release-date-item__country-name a")]
    release_dates = [i.text for i in soup.select(".release-date-item__date")]
    df = pd.DataFrame()
    df["country"] = countries
    df["release_date"] = release_dates
    grouped = df.groupby("release_date")
    n_countries = grouped["country"].count().max()
    most_release_dates = grouped["country"].count().idxmax()
    return "共有 {} 個國家在 {} 上映 Avengers: Endgame (2019)".format(n_countries, most_release_dates)

In [5]:
answer_two()

'共有 31 個國家在 24 April 2019 上映 Avengers: Endgame (2019)'